# Test on Velodyne data

In [ ]:
%matplotlib widget
import numpy as np
import os
import time
import plotly.graph_objects as go

from planeslam.general import downsample, adaptive_downsample
from planeslam.mesh import LidarMesh
from planeslam.scan import Scan, velo_pc_to_scan
from planeslam.clustering import cluster_mesh_graph_search, plot_clusters
from planeslam.extraction import scan_from_clusters, planes_from_clusters
from planeslam.general import pc_plot_trace
from planeslam.geometry.util import quat_to_R
from planeslam.point_cloud import velo_preprocess

%load_ext autoreload
%autoreload 2

In [ ]:
np.set_printoptions(suppress=True)

Flight room

In [ ]:
# Read in point cloud data
pcpath = os.path.join(os.getcwd(),'..', '..', 'data', 'velodyne', '8_20_2022', 'flightroom', 'run_7', 'pcs')
PCs = []
#len(os.listdir(pcpath))
#select_idxs = np.arange(0, 1000, 2)
select_idxs = np.arange(0, len(os.listdir(pcpath)), 5)
for i in select_idxs:  
    filename = os.path.join(pcpath, 'pc_'+str(i)+'.npy')
    PC = np.load(filename)
    PCs.append(PC)

In [ ]:
# Read in pose data
posepath = os.path.join(os.getcwd(),'..', '..', 'data', 'velodyne', '8_20_2022', 'flightroom', 'run_7', 'poses')
poses = []
for i in select_idxs:  
    filename = os.path.join(posepath, 'pose_'+str(i)+'.npy')
    pose = np.load(filename)
    poses.append(pose)

In [ ]:
idx = 13
PC_orig = PCs[idx]
pose = poses[idx]

In [ ]:
# data = go.Scatter3d(x=PC_orig[:,0], y=PC_orig[:,1], z=PC_orig[:,2],
#         mode='markers', marker=dict(size=2, color=np.arange(len(PC_orig))))
# fig = go.Figure(data=data)
# fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
# fig.show()

In [ ]:
# Pre-process 
PC = velo_preprocess(PC_orig, pose)
# Downsample
scan = velo_pc_to_scan(PC)
scan.remove_small_planes(area_thresh=0.1)
scan.reduce_inside(p2p_dist_thresh=0.1)

In [ ]:
fig = go.Figure(data=[pc_plot_trace(PC)]+scan.plot_trace())
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
# Pre-process 
PC = velo_preprocess(PC_orig, pose)
# Downsample
PC = adaptive_downsample(PC, factor=5)

In [ ]:
fig = go.Figure(data=pc_plot_trace(PC))
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
# Create the mesh
mesh = LidarMesh(PC)
mesh.prune(edge_len_lim=0.5)
# start_time = time.time()
# # Cluster the mesh with graph search
# clusters, avg_normals = cluster_mesh_graph_search(mesh)
# print("elapsed time: ", time.time() - start_time)

In [ ]:
mesh.smooth_laplacian()

In [ ]:
# Plot mesh
fig = go.Figure(data=mesh.plot_trace()+[pc_plot_trace(PC)])
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
clusters, avg_normals = cluster_mesh_graph_search(mesh, min_cluster_size=50)

In [ ]:
# Plot clusters
plot_clusters(PC, mesh, clusters)

In [ ]:
planes, basis = planes_from_clusters(mesh, clusters, avg_normals)
scan = Scan(planes, basis)

In [ ]:
fig = go.Figure(data=mesh.plot_trace()+scan.plot_trace())
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
from planeslam.clustering import mesh_cluster_pts, sort_mesh_clusters

clusters, avg_normals = sort_mesh_clusters(clusters, avg_normals)
i = 0
n = avg_normals[i][:,None]
c = clusters[i]
cluster_pts = mesh_cluster_pts(mesh, c)  # Extract points from cluster

In [ ]:
normals_arr = np.asarray(avg_normals)
normal_dirs = np.argmax(np.abs(normals_arr), axis=1)  # normal directions (x,y,z)

In [ ]:
ground_clusters = [clusters[i] for i in range(len(clusters)) if (normal_dirs==2)[i]]

In [ ]:
from planeslam.general import normalize


ground_normal = normalize(np.mean(normals_arr[normal_dirs==2], axis=0))

In [ ]:
ground_cluster = [i for c in ground_clusters for i in c]

In [ ]:
wall_clusters = [clusters[i] for i in range(len(clusters)) if (normal_dirs!=2)[i]]

In [ ]:
wall_normals = list(normals_arr[normal_dirs!=2])

In [ ]:
clusters = [ground_cluster] + wall_clusters

In [ ]:
avg_normals = [ground_normal] + wall_normals

In [ ]:
len(clusters)

In [ ]:
a = np.array([[0,1],[1,2],[2,7]])
b = np.array([0,1,2,3])
np.all(np.isin(a,b), axis=1)

In [ ]:
idxs[cts!=1]

In [ ]:
# Plot the points 
fig = go.Figure(data=pc_plot_trace(cluster_pts))
fig.update_layout(width=1000, height=600, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
# Find ground plane - largest cluster with largest normal component in z
# (assumes pitch/roll is < 45 degrees)
normals_arr = np.asarray(avg_normals)
normal_dirs = np.argmax(np.abs(normals_arr), axis=1)  # normal directions (x,y,z)
ground_normal = normals_arr[normal_dirs==2][0]

# Group normals into x and y
x_normals = normals_arr[normal_dirs==0]
y_normals = normals_arr[normal_dirs==1]

# Find "best fit" x and y basis vectors

In [ ]:
np.mean(x_normals, axis=1)

In [ ]:
np.mean(np.sign(x_normals[:,0])[:,None] * x_normals, axis=0)